In [11]:
# !pip install python-dotenv numpy faiss-gpu pandas

In [12]:
# !pip install sentence_transformers

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
import torch

In [ ]:
# from google.colab import userdata
# hf_token = userdata.get('HF_TOKEN')

In [5]:
load_dotenv()

hf_token = os.getenv("HF_TOKEN")

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
print(torch.version.cuda)

12.4


In [6]:
# 1. csv 로드 및 사용할 llm 모델 로드
# cosmetics_df = pd.read_csv("cosmetics.csv", encoding='utf-8')
# ingredients_df = pd.read_csv("../../data/preprocessed_ingredients.csv", encoding='utf-8')
ingredients_df = pd.read_csv("./data/preprocessed_ingredients.csv", encoding='utf-8')

model = SentenceTransformer("all-MiniLM-L6-v2")
if model.tokenizer.pad_token is None:
    model.tokenizer.pad_token = model.tokenizer.eos_token

print('✅ 1. csv 로드 및 사용할 llm 모델 로드')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ 1. csv 로드 및 사용할 llm 모델 로드


In [ ]:
# cpu 코어 수 조정
# torch.set_num_threads(4)

In [7]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [8]:
# 2. 벡터화할 텍스트 추출
ingredient_texts = ingredients_df['description']
# product_texts = cosmetics_df.apply(
#     lambda row: f"{row['brand']} {row['product_name']} 성분: {row['ingredient']} 리뷰: {row['reviews']} 사용법: {row['usage']}", axis=1
# ).tolist()

all_texts = ingredient_texts
# all_texts = ingredient_texts + product_texts
all_embeddings = model.encode(
    all_texts,
    batch_size=128,
    convert_to_numpy=True,
    show_progress_bar=True
)

print('✅ 2. 벡터화할 텍스트 추출')

Batches:   0%|          | 0/184 [00:00<?, ?it/s]

✅ 2. 벡터화할 텍스트 추출


In [10]:
len(all_embeddings)

23544

In [11]:
# 3. FAISS 인덱스 생성
embedding_dim = all_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(all_embeddings)

print('✅ 3. FAISS 인덱스 생성')

✅ 3. FAISS 인덱스 생성


In [12]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f72304a7030> >

In [13]:
# 4. 메타데이터 저장
metadata = []

# 성분 설명 메타
for i, row in ingredients_df.iterrows():
    metadata.append({
        "type": "ingredient",
        "name": row['ingredient'],
        "description": row['description']
    })

# 화장품 정보 메타
# for i, row in cosmetics_df.iterrows():
#     metadata.append({
#         "type": "cosmetic",
#         "product_name": row['product_name'],
#         "brand": row['brand'],
#         "ingredients": row['ingredients'],
#         "reviews": row['review'],
#         "usage": row['usage'],

#     })

print('✅ 4. 메타데이터 저장')

✅ 4. 메타데이터 저장


In [16]:
# 5. 인덱스와 메타 데이터 저장
# faiss.write_index(index, "cosmetic_ingredient.index")
faiss.write_index(index, "ingredient.index")

with open("ingredient_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("✅ 5. 인덱스와 메타 데이터 저장")


✅ 5. 인덱스와 메타 데이터 저장


In [21]:
query = "잡티 제거 효과를 지닌 성분"
query_vec = model.encode([query])
D, I = index.search(query_vec, k=5)

for idx, dist in zip(I[0], D[0]):
    print(f"메타데이터: {metadata[idx]}")
    print(f"유사도 점수(Distance): {dist}")

메타데이터: {'type': 'ingredient', 'name': '팥꽃나무싹추출물', 'description': '팥꽃나무싹추출물은 피부 진정 및 항염 효과로 피부 관리 제품에 사용됩니다. 항산화 성분이 있어 노화 방지에 도움이 될 수 있습니다. 그러나 일부 피부에 자극을 줄 수 있어 사용 전 패치 테스트가 권장됩니다.'}
유사도 점수(Distance): 0.39007627964019775
메타데이터: {'type': 'ingredient', 'name': '리날룰리페룸녹나무잎뿌리오일', 'description': '리날룰리페룸녹나무잎뿌리오일은 주로 항균 및 항염 효과로 알려져 있습니다. 피부 진정 및 보습 효과를 제공하며, 향을 강화하는 데 사용됩니다. 개별 민감성에 따라 피부 자극이 있을 수 있으니 사용 전 패치 테스트가 권장됩니다.'}
유사도 점수(Distance): 0.3912869691848755
메타데이터: {'type': 'ingredient', 'name': '스위트시슬리추출물', 'description': '스위트시슬리추출물은 피부 진정 및 항염 효과가 있어 주로 화장품에 사용됩니다. 또한 항산화 성분이 함유되어 피부 건강을 지원하고 노화 방지에 도움을 줄 수 있습니다. 알레르기 반응을 일으킬 수 있으므로 피부 테스트가 권장됩니다.'}
유사도 점수(Distance): 0.4183346927165985
메타데이터: {'type': 'ingredient', 'name': '에우칼륍투스 스미티이잎오일', 'description': '에우칼립투스 스미티이잎오일은 피부 진정 및 항염 효과로 주로 사용되며, 상쾌한 향을 제공하여 아로마테라피에도 활용됩니다. 다만, 민감한 피부에는 자극이 있을 수 있어 테스트 후 사용이 권장됩니다.'}
유사도 점수(Distance): 0.4265833795070648
메타데이터: {'type': 'ingredient', 'name': '바실러스/왕벚나무잎추출물발효여과물', 'description': '바실러스/왕벚나무잎